In [1]:
%load_ext autoreload

In [16]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
import logging
from scipy.linalg import block_diag

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               FindPolytopeGivenLagrangianOptions, 
                                               FindSeparationCertificateGivenPolytopeOptions, 
                                               BilinearAlternationOptions,
                                               BinarySearchOptions,
                                               SeparatingPlaneOrder,
                                               EllipsoidMarginCost)

In [5]:
import pydrake.multibody.rational_forward_kinematics as rat_fwd
from pydrake.all import RationalForwardKinematicsOld

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [6]:
# #construct our robot
# builder = DiagramBuilder()
# plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
# parser = Parser(plant)
# oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

# box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

# models = []
# models.append(parser.AddModelFromFile(box_asset))
# models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
# models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))
# locs = [[0.,0.,0.],
#         [0,1,0.85],
#         [0,-1,0.55]]
# plant.WeldFrames(plant.world_frame(), 
#                  plant.GetFrameByName("base", models[0]),
#                  RigidTransform(locs[0]))

# t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
# t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
# plant.WeldFrames(plant.world_frame(), 
#                  plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
#                  t1)
# plant.WeldFrames(plant.world_frame(), 
#                  plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
#                  t2)


# plant.Finalize()
# idx = 0
# q0 = [0.0, 0.0]
# val = 1.7
# q_low  = [-val, -val, 0]
# q_high = [val, val,  0]
# # set the joint limits of the plant
# for model in models:
#     for joint_index in plant.GetJointIndices(model):
#         joint = plant.get_mutable_joint(joint_index)
#         if isinstance(joint, RevoluteJoint):
#             joint.set_default_angle(q0[idx])
#             joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
#             idx += 1
        


In [7]:
boxes_urdf = """
<robot name="boxes">
  <link name="fixed">
    <collision name="right">
      <origin rpy="0 0 0" xyz="2 0 0"/>
      <geometry><box size="1 1 5"/></geometry>
    </collision>
    <visual>
        <geometry><box size="1 1 5"/></geometry>
        <origin rpy="0 0 0" xyz="2 0 0"/>
    </visual>
    <collision name="left">
      <origin rpy="0 0 0" xyz="-2 0 0"/>
      <geometry><box size="1 1 5"/></geometry>    
    </collision>
    <visual>
        <geometry><box size="1 1 5"/></geometry>
        <origin rpy="0 0 0" xyz="-2 0 0"/>
    </visual>
  </link>
  <joint name="fixed_link_weld" type="fixed">
    <parent link="world"/>
    <child link="fixed"/>
  </joint>
  <link name="movable">
    <collision name="center">
      <geometry><box size="1 1 1"/></geometry>
      <origin rpy="0 0 0" xyz="0 0 2"/>
    </collision>
    <visual>
        <geometry><box size="1 1 1"/></geometry>
        <origin rpy="0 0 0" xyz="0 0 1"/>
    </visual>
  </link>
  <joint name="movable" type="revolute">
    <axis xyz="0 1 0"/>
    <limit lower="-3" upper="3"/>
    <parent link="world"/>
    <child link="movable"/>
  </joint>
</robot>
"""

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.AddModelsFromString(boxes_urdf, "urdf")
plant.Finalize()

In [8]:
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)


# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())


q_low = plant.GetPositionLowerLimits()
q_high = plant.GetPositionUpperLimits()

#compute limits in t-space
limits_s = []
for q in [q_low, q_high]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
# visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

context = visualizer.diagram_context
diagram = visualizer.diagram

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001
/home/amice/Documents/coding_projects/drake/C_Iris_Examples/iris_plant_visualizer.py:70: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(self.diagram_context)


## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [9]:
sliders = [widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description=f'q{i}') for i in range(plant.num_positions())]


q = q_star
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=3.0, min=-3.0)

In [10]:
# # filter fused joints self collisions so they don't interfere with collision engine
# digaram = visualizer.diagram
# context = visualizer.diagram_context
# sg_context = scene_graph.GetMyContextFromRoot(context)
# inspector = scene_graph.model_inspector()

# pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)
# right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
# left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

# right_sweeper_fused_col_geom = right_sweeper_gids[2:]
# right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
# left_sweeper_fused_col_geom = left_sweeper_gids[4:]
# left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

# right_sweeper_end_gid = right_sweeper_gids[-1]
# left_sweeper_end_gid = left_sweeper_gids[-1]
# id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
#                        ]
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [11]:
# Some seedpoints
# seed_points_q = np.array([   [0.0, 0],
#                               [0.7, -0.9],
#                               [-0.5, -0.5],
#                               [0.4,-1.3]
#                               ])
seed_points_q = np.array([[0]])

seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], q_star)\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
colors_dict = {
    0: (144,144,144),
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}

In [12]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 1)


find_polytope_given_lagrangian_option = FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = False

find_separation_certificate_given_polytope_options = FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = True

bilinear_alternation_options = BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 20
bilinear_alternation_options.convergence_tol = 1e-5
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 100
binary_search_options.max_iter = 50


cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

In [13]:
# set up old certifier
cspace_free_region_certifier = rat_fwd.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rat_fwd.SeparatingPlaneOrderOld.kAffine,
                                   rat_fwd.CspaceRegionType.kGenericPolytope)

## Seeding and Certifying with a Stronger Heuristic
We have also implemented another, strong heuristic for proposing good initial regions based on non-linear optimization. See Appendix TODO of our paper TODO for details

In [15]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, q_star)
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context), 
                                         q_star, 
                                         iris_options)
    iris_regions.append(r)
#     iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid)
    

if do_viz:
    regions_to_viz = iris_regions
    if q_star.shape[0] == 1:
        for i, r in regions_to_viz:
            Ay = np.array([[1],[-1]])
            by = np.array([[5],[5]])
            tmp = HPolyhedron(block_diag(r.A(), Ay), n)
    visualizer.plot_regions(iris_regions,
                            region_suffix='_iris_regions',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

TypeError: cannot unpack non-iterable pydrake.geometry.optimization.HPolyhedron object

In [39]:
limits_s

[array([-1.13833271, -1.13833271]), array([1.13833271, 1.13833271])]

In [37]:
q_high

array([1.7, 1.7])

### These regions tend to be very large, but typically are not completely collision free. We can use the binary search method to find a uniform shrinking of these regions to prove their safety and then again improve them with bilinear alternations.

In [13]:
binary_search_options_for_iris = BinarySearchOptions()
binary_search_options_for_iris.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options_for_iris.max_iter = 50

In [14]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
bin_sect_regions = []
for i, (s, initial_region) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    certificate = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    initial_region.MaximumVolumeInscribedEllipsoid().center(), 
                                                    binary_search_options_for_iris)
    bin_sect_regions.append(HPolyhedron(certificate.C, certificate.d))
    binary_search_region_certificates_for_iris[tuple(s)] = [HPolyhedron(certificate.C, certificate.d)]

INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.505 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.2575 is feasible


starting seedpoint 1/4


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.38125 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.31937499999999996 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.2884375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.30390625 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.311640625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.30777343749999997 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.30583984374999995 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.30487304687499994 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.505 is feasible


starting seedpoint 2/4


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.7525 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.87625 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.938125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.9690624999999999 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.95359375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.961328125 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.9574609375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.9593945312500001 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.9584277343750001 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.505 is infeasible


starting seedpoint 3/4


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.2575 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.38125 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.31937499999999996 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.35031249999999997 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.36578124999999995 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.358046875 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.35417968749999995 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.35611328124999997 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.35514648437499996 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.505 is infeasible


starting seedpoint 4/4


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.2575 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.38125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.443125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.4740625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.45859375 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.450859375 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.4469921875 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.44892578125000004 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=0.44795898437500004 is infeasible


In [15]:
if do_viz:
    visualizer.plot_regions(bin_sect_regions, ellipses=None,
                            region_suffix='_new_bin_search',
                            wireframe = False)

## Now compare to the old methods

In [17]:
i = 0
bisect_regions = []
bisect_solutions = []
for i in range(len(iris_regions)):
    print(i)
    region_to_certify = iris_regions[i]
    seed_point = seed_points[i,:]


    binary_search_options_old = rat_fwd.BinarySearchOption()
    binary_search_options_old.epsilon_max = 0 # it is very unlikely that we can find a uniform expansion of the current region
    binary_search_options_old.max_iters = 15
    # speed up the bisection search by taking non-uniform steps when possible
    binary_search_options_old.search_d = False #set false for fairness
    # find the smallest e such that At <= b + e1 still contains our seed point.
    binary_search_options_old.epsilon_min = rat_fwd.FindEpsilonLower(region_to_certify.A(),
                                                                                     region_to_certify.b(),
                                                                                     limits_s[0], limits_s[1],
                                                                                     seed_point)
    #use as many threads as possible to speed up computation
#     binary_search_options.num_threads = 1

    certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
                                                                     q_star[:2],
                                                                     set(),
                                                                     region_to_certify.A(),
                                                                     region_to_certify.b(),
                                                                     binary_search_options_old, 
                                                                     solver_options,
                                                                     seed_point)
    bisect_solutions.append(certified_region_contraction_solution)
    certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
                                               certified_region_contraction_solution.d)
    
#     bisect_regions.append(promote_region_to_3d(certified_region_contraction))
    bisect_regions.append(certified_region_contraction)
if do_viz:
    visualizer.plot_regions(bisect_regions, ellipses=None,
                            region_suffix='_old_bin_search',
                            wireframe = False)

INFO:drake:Lagrangian SOS takes 0.5059952549636364 seconds
INFO:drake:max(power(det(P), 1/2))=0.23758606874481192, solver_time 0.002695586998015642
INFO:drake:Lagrangian SOS takes 0.40055954398121685 seconds


0
1


INFO:drake:max(power(det(P), 1/2))=0.15437670455929708, solver_time 0.005525437067262828
INFO:drake:Lagrangian SOS takes 0.6465820700395852 seconds
INFO:drake:max(power(det(P), 1/2))=0.40893539393457645, solver_time 0.003291556960903108
INFO:drake:Lagrangian SOS takes 0.3570827750954777 seconds


2
3


INFO:drake:max(power(det(P), 1/2))=0.40258822396387683, solver_time 0.0051983140874654055
